## Assigment 6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from a6_utils import drawEllipse

### Exercise 1

### A

A = (3, 4), B = (3, 6), C = (7, 6), and D = (6, 4)  

mean value of X = (3 + 3 + 7 + 6, 4 + 6 + 6 + 4) / 4 = (19, 20) / 4 = (4.75, 5)  

centered X matrix:

$$\begin{bmatrix} -1.75 & -1.75 & 2.25 & 1.25 \\ -1 & 1 & 1 & -1 \end{bmatrix}$$

covariance matrix: 

$$\begin{bmatrix} \frac{17}{4} & \frac{1}{3} \\ \frac{1}{3} & \frac{4}{3} \end{bmatrix}$$


### B

In [ ]:
def mean(X):
    return np.mean(X, axis=1).reshape(-1, 1)

def centered(X):
    return X - mean(X)

def covariance_matrix(X):
    X_centered = centered(X)
    N = X_centered.shape[1] - 1
    return np.dot(X_centered, X_centered.T) / N

def main_components(X):
    X_covariance = covariance_matrix(X)
    main_directions, importance, _ = np.linalg.svd(X_covariance)
    return main_directions, importance


In [ ]:
points = np.fromfile('./assigments/assignment6/data/points.txt', dtype=float, sep=' ').reshape(-1, 2).T
points

In [ ]:
fig = plt.figure(figsize=(5,5))

plt.scatter(points[0], points[1], color='blue')
drawEllipse(mean(points), covariance_matrix(points), n_std=1)

plt.xlim(-2, 8)
plt.ylim(-2, 8)

### C

In [ ]:
main_directions, importance = main_components(points)
main_directions, importance

In [ ]:
fig = plt.figure(figsize=(5,5))

plt.scatter(points[0], points[1], color='blue')
drawEllipse(mean(points), covariance_matrix(points), n_std=1)

plt.xlim(-2, 8)
plt.ylim(-2, 8)

mean_point = mean(points)

center = mean_point
end_red = mean_point + main_directions[:, [0]] * np.sqrt(importance[0])
end_green = mean_point + main_directions[:, [1]] * np.sqrt(importance[1])
plt.plot([center[0], end_red[0]], [center[1], end_red[1]], color='red')
plt.plot([center[0], end_green[0]], [center[1], end_green[1]], color='green')

### D

In [ ]:
# plot the eigenvalues
plt.bar([0, 1], importance/np.sum(importance), color=['red', 'green'])

### E

In [ ]:
fig = plt.figure(figsize=(5,5))

pca_points = np.dot(main_directions.T, centered(points))
plt.scatter(pca_points[0], pca_points[1], color='blue')

plt.xlim(-5, 5)
plt.ylim(-5, 5)

In [ ]:
pca_projected_points = pca_points.copy()
pca_projected_points[1, :] = 0

reconstructed_points = np.dot(main_directions, pca_projected_points) + mean(points)

In [ ]:
plt.scatter(reconstructed_points[0], reconstructed_points[1], color='red')
plt.scatter(points[0], points[1], color='blue')

### F

In [ ]:
q_point = np.array([[6], [6]])

closes_point = np.argmin(np.linalg.norm(points - q_point, axis=0))

pca_q_point = np.dot(main_directions.T, q_point - mean(points))
pca_q_point[1] = 0

reconstructed_q_point = np.dot(main_directions, pca_q_point) + mean(points)

In [ ]:
plt.scatter(points[0], points[1], color='blue')
plt.scatter(q_point[0], q_point[1], color='red')

plt.scatter(reconstructed_points[0], reconstructed_points[1], color='green')
plt.scatter(reconstructed_q_point[0], reconstructed_q_point[1], color='purple')

plt.plot([q_point[0], reconstructed_q_point[0]], [q_point[1], reconstructed_q_point[1]], color='black')
for i in range(points.shape[1]):
    plt.plot([points[0, i], reconstructed_points[0, i]], [points[1, i], reconstructed_points[1, i]], color='black')

## Exercise 2

### A

In [ ]:
def dual_pca(X):
    N = X.shape[1]
    X_centered = centered(X)
    ipm = np.dot(X_centered.T, X_centered) / N
    main_directions, importance, _ = np.linalg.svd(ipm)
    importance += 10**-15
    main_directions = np.dot(X_centered, main_directions) / np.sqrt(N * importance)
    return main_directions, importance

In [ ]:
dp_main_directions, dp_importance = dual_pca(points)
main_directions, dp_main_directions[:2, :2]

### B

In [ ]:
# offset
offset = mean(points)

# project points
projected_points = np.dot(dp_main_directions.T, points - offset)

# reconstructed points
reconstructed_points = np.dot(dp_main_directions, projected_points) + offset

# plot
fig = plt.figure(figsize=(5,5))
plt.scatter(points[0], points[1], color='red', marker='|', s=50)
plt.scatter(reconstructed_points[0], reconstructed_points[1], color='black', marker='_', s=50)

## Exercise 3

### A

In [ ]:
import os
import cv2
def load_space(directory):
    # load each image as a grayscale vector
    images = []
    image_shape = None
    for image_name in os.listdir(directory):
        image = cv2.imread(os.path.join(directory, image_name), cv2.IMREAD_GRAYSCALE)
        images.append(image.flatten() / 255)
        image_shape = image.shape
    return np.array(images).T, image_shape

In [ ]:
space_one, one_sahpe = load_space('./assigments/assignment6/data/faces/1')
space_one.shape, one_sahpe

In [ ]:
space_two, two_shape = load_space('./assigments/assignment6/data/faces/2')
space_two.shape, two_shape

In [ ]:
space_three, three_shape = load_space('./assigments/assignment6/data/faces/3')
space_three.shape, three_shape

### B

In [ ]:
def transofrmation_parameters(X):
    main_directions, importance = dual_pca(X)
    m = mean(X)
    return main_directions, m

In [ ]:
space_one_eginvectors, space_one_means = transofrmation_parameters(space_one)

In [ ]:
N = 5
fig = plt.figure(figsize=(10, 10))

for i in range(N):
    plt.subplot(1, N, i+1)
    plt.imshow(space_one_eginvectors[:, i].reshape(*one_sahpe), cmap='gray')

In [ ]:
def project(X, eginvectors, mean):
    return np.dot(eginvectors.T, X - mean)
    
def reconstruct(X, eginvectors, mean):
    return np.dot(eginvectors, X) + mean

In [ ]:
first_image = space_one[:, [0]]
first_image.shape

In [ ]:
projected = project(first_image, space_one_eginvectors, space_one_means)
projected.shape

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].imshow(first_image.reshape(*one_sahpe), cmap='gray')

first_image_without_4074 = first_image.copy()
first_image_without_4074[4074] = 0
ax[1].imshow(first_image_without_4074.reshape(*one_sahpe), cmap='gray')

projected_without_main = projected.copy()
projected_without_main[0] = 0
reconstructed_without_main = reconstruct(projected_without_main, space_one_eginvectors, space_one_means)
ax[2].imshow(reconstructed_without_main.reshape(*one_sahpe), cmap='gray')

In [ ]:
len(np.argwhere(first_image != reconstructed_without_main))

### C

In [ ]:
projected_step_reduce = projected.copy()

fig, ax = plt.subplots(1, 6, figsize=(15, 5))

for i in range(5, -1, -1):
    projected_step_reduce[2**i:] = 0
    reconstructed_step_reduce = reconstruct(projected_step_reduce, space_one_eginvectors, space_one_means)
    ax[5-i].imshow(reconstructed_step_reduce.reshape(*one_sahpe), cmap='gray')

### D

In [ ]:
space_two_eginvectors, space_two_means = transofrmation_parameters(space_two)

In [ ]:
projected_mean = project(space_two[:, [0]], space_two_eginvectors, space_two_means)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(space_two_means.reshape(*two_shape), cmap='gray')

ax[1].imshow(reconstruct(projected_mean, space_two_eginvectors, space_two_means).reshape(*two_shape), cmap='gray')

In [ ]:
def reconstruct_with_weight(X, i, scale=3000, loops=1, samples=100):
    local_egin = space_two_eginvectors.copy()
    for _ in range(loops):
        value = np.sin(np.linspace(0, 2*np.pi, num=samples))
        for v in value:
            local_egin[:, :i] = v * scale
            yield reconstruct(X, local_egin, space_two_means)

In [ ]:
from time import sleep
fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].imshow(space_two_means.reshape(*two_shape), cmap='gray')

ax[1].imshow(reconstruct(projected_mean, space_two_eginvectors, space_two_means).reshape(*two_shape), cmap='gray')

samples = 1
generator = reconstruct_with_weight(projected_mean, 10, loops=3, samples=samples, scale=6000)
for img in generator:
    ax[2].imshow(img.reshape(*two_shape), cmap='gray')
    plt.draw()
    plt.pause(1 / samples)


### E

In [ ]:
elephants = cv2.imread('./assigments/assignment6/data/elephant.jpg', cv2.IMREAD_GRAYSCALE)
elephants.shape

In [ ]:
elephants_vector = elephants.reshape(-1, 1)

In [ ]:
reconstructed_elephants = reconstruct(project(elephants_vector, space_one_eginvectors, space_one_means), space_one_eginvectors, space_one_means)

In [ ]:
space_one_reduced = space_one_eginvectors.copy()
space_one_reduced[:, 100:] = 0

space_one_main = space_one_eginvectors.copy()
space_one_main[:, :100] = 0

In [ ]:
reconstructed_elephants_reduced = reconstruct(project(elephants_vector, space_one_reduced, space_one_means), space_one_reduced, space_one_means)
reconstructed_elephants_main = reconstruct(project(elephants_vector, space_one_main, space_one_means), space_one_main, space_one_means)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(10, 5))

ax[0].imshow(elephants, cmap='gray')

ax[1].imshow(reconstructed_elephants.reshape(*elephants.shape), cmap='gray') # same because operations are reversible

ax[2].imshow(reconstructed_elephants_main.reshape(*elephants.shape), cmap='gray')

ax[3].imshow(reconstructed_elephants_reduced.reshape(*elephants.shape), cmap='gray')

### G

In [ ]:
space_three_eginvectors, space_three_means = transofrmation_parameters(space_three)

In [ ]:
projected_space_one = project(space_one, space_one_eginvectors, space_one_means)
projected_space_two = project(space_two, space_two_eginvectors, space_two_means)
projected_space_three = project(space_three, space_three_eginvectors, space_three_means)

In [ ]:
space_one_mean = mean(projected_space_one)
space_two_mean = mean(projected_space_two)
space_three_mean = mean(projected_space_three)
space_mean = (space_one_mean + space_two_mean + space_three_mean) / 3

In [ ]:
centered_means = [space_one_mean - space_mean, space_two_mean - space_mean, space_three_mean - space_mean]
centered_classes = [projected_space_one - space_one_mean, projected_space_two - space_two_mean, projected_space_three - space_three_mean]

In [ ]:
def between_class_scatter(centered_means):
    SB = np.zeros((len(centered_means[0]), len(centered_means[0])))
    for cm in centered_means:
        SB += len(cm) * (cm @ cm.T)
    return SB

def within_class_scatter(centered_classes):
    SW = np.zeros((len(centered_classes[0]), len(centered_classes[0])))
    for cc in centered_classes:
        SW += cc @ cc.T
    return SW


In [ ]:
SB = between_class_scatter(centered_means)
SW = within_class_scatter(centered_classes)

In [ ]:
SB.shape, SW.shape

In [ ]:
eigvecs, egival, _ = np.linalg.svd(np.linalg.inv(SW) @ SB)

In [ ]:
lda_projected_one = project(projected_space_one, eigvecs, space_one_mean)
lda_projected_two = project(projected_space_two, eigvecs, space_two_mean)
lda_projected_three = project(projected_space_three, eigvecs, space_three_mean)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].scatter(projected_space_one[0], projected_space_one[1], label='space one')
axes[0].scatter(projected_space_two[0], projected_space_two[1], label='space two')
axes[0].scatter(projected_space_three[0], projected_space_three[1], label='space three')

axes[1].scatter(lda_projected_one[0], lda_projected_one[1], label='space one')
axes[1].scatter(lda_projected_two[0], lda_projected_two[1], label='space two')
axes[1].scatter(lda_projected_three[0], lda_projected_three[1], label='space three')